In [1]:
# Imports
import config
import ccxt
import schedule
import pandas as pd
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Fetch exchange
exchange = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': config.TESTNET_API,
    'secret': config.TESTNET_SECRET,
    #'verbose': True,
})

# Load markets so they populate data faster
markets = exchange.load_markets()

# Specify paper trading environment
sandbox = exchange.set_sandbox_mode(True)

In [ ]:
# Test fetch ohlcvc call
exchange.fetch_ohlcvc('LUNA/USD:USD', timeframe='1m', limit=50)

In [ ]:
# Test create order call
exchange.create_order('LUNA/USD:USD', 'market', 'buy', 100, None)

In [ ]:
# Test stop order placement call
buy_stop_order = exchange.create_order('LUNA/USD:USD', 'Stop', 'sell', 100, None, {'stopPx': 50.0})

In [ ]:
# Get buy stop order id
buy_stop_id = buy_stop_order['info']['orderID']

# Cancel buy stop order
exchange.cancel_order(buy_stop_id, 'LUNA/USD:USD')

In [ ]:
# Create EMA used for signal
def ema(df):
    ema = df['close'].ewm(com=20).mean()
    return ema

In [ ]:
# Create trigger function to find signals and take trades
def trigger(df):
    # Every time the function is run, it will print checking for signals and print the tail rows. 
    print('Checking for buy and sell signals')
    print(df.tail(50))

    # Generate last and previous row index for buy/sell signal functionality
    last_row = len(df.index) - 1
    prev = last_row - 1

    # Check for open orders, generate actual buy and sell stop prices, set buy and sell stop price to 0 as flag for later. 
    open_orders = exchange.fetch_open_orders('LUNA/USD:USD')
    buy_stop = df['ema'][prev] - (df['ema'][prev] * .01)
    sell_stop = df['ema'][prev] + (df['ema'][prev] * .01)
    buy_stop_price = 0
    sell_stop_price = 0

    # Check for open orders, if no open orders and buy/sell signal triggered, take trade
    if not open_orders:
        # Long signal
        if df['close'][prev] > df['ema'][prev]:  
            print('Long signal triggered')
            
            # Create buy order
            buy_order = exchange.create_order('LUNA/USD:USD', 'market', 'buy', 100, None)
            print(buy_order)
            
            # Create buy stop loss and set stop loss price to buy_stop variable above
            buy_stop_order = exchange.create_order('LUNA/USD:USD', 'Stop', 'sell', 100, None, {'stopPx': buy_stop})
            print(buy_stop_order)

            # Get id and stop price to access order and cancel later
            buy_stop_id = buy_stop_order['info']['orderID']
            print('Buy stop ID is: ', buy_stop_id)

            buy_stop_price = buy_stop_order['stopPrice']
            print('Buy stop price is:', buy_stop_price)

            # Return values we will need for next iteration
            # return buy_stop_id, buy_stop_price, buy_or_sell

        # Elif short signal, create short order, short stop loss, and get id/stop price
        elif df['close'][prev] < df['ema'][prev]:
            print('Short signal triggered')

            sell_order = exchange.create_order('LUNA/USD:USD', 'market', 'sell', 100, None)
            print(sell_order)

            sell_stop_order = exchange.create_order('LUNA/USD:USD', 'Stop', 'buy', 100, None, {'stopPx': sell_stop})
            print(sell_stop_order)

            sell_stop_id = sell_stop_order['info']['orderID']
            print('Sell stop ID is:', sell_stop_id)

            sell_stop_price = sell_stop_order['stopPrice']
            print('Sell stop price is:', sell_stop_price)
        
    open_orders = exchange.fetch_open_orders('LUNA/USD:USD')
    # If there are open orders, if buy stop price != 0 and buy stop price is less than buy stop value, cancel stop loss and create new one
    # Vice versa for sell order
    
    # Get data from current order
    if open_orders:
        """
        This can be confusing. When checking for open orders, it will return the current conditional order
        that has not yet been triggered. So if you have a buy stop, the side of the order will be Sell since 
        you sell the same amount of contracts that you bought to close the order. Vice versa for sell stop orders. 
        """
        if open_orders[0]['info']['side'] == 'Sell':
            buy_stop_id = open_orders[0]['info']['orderID']
            buy_stop_price = open_orders[0]['stopPrice']
            print(f"Current Trade Type: {open_orders[0]['info']['side']}")
            print(f"buy Stop = {buy_stop} buy Stop Price = {buy_stop_price}")
        elif open_orders[0]['info']['side'] == 'Buy':
            sell_stop_id = open_orders[0]['info']['orderID']
            sell_stop_price = open_orders[0]['stopPrice']
            print(f"Current Trade Type: {open_orders[0]['info']['side']}")
            print(f"sell_Stop = {sell_stop} sell_Stop Price = {sell_stop_price}")
        
        # This is where the flag from above comes in to execute the cancel order.
        
        # If buy stop has been generated, then buy stop price will be the updated buy stop price, not 0.
        # If our buy stop order price is less than what we actaully want the buy stop to be once new data is brought in, then we update it. 
        if buy_stop_price != 0 and buy_stop_price < buy_stop: 
            print('Buy stop changed.')
            
            # Cancel buy stop order
            cancel_buy_stop = exchange.cancel_order(buy_stop_id, 'LUNA/USD:USD')
            print(cancel_buy_stop)
            
            # Create new buy stop order
            buy_stop_order = exchange.create_order('LUNA/USD:USD', 'Stop', 'sell', 100, None, {'stopPx': buy_stop})
            print(buy_stop_order)
            
            # Get new buy stop id and price
            buy_stop_id = buy_stop_order['info']['orderID']
            print(buy_stop_id)

            buy_stop_price = buy_stop_order['stopPrice']
            print('Updated buy stop is: ', buy_stop_price)
        
        # Same functionality from above applies to this block
        elif sell_stop_price != 0 and sell_stop_price > sell_stop:
            print('Sell stop changed')

            cancel_sell_stop = exchange.cancel_order(sell_stop_id, 'LUNA/USD:USD')
            print(cancel_sell_stop)

            sell_stop_order = exchange.create_order('LUNA/USD:USD', 'Stop', 'buy', 100, None, {'stopPx': sell_stop})
            print(sell_stop_order)

            sell_stop_id = sell_stop_order['info']['orderID']
            print(sell_stop_id)

            sell_stop_price = sell_stop_order['stopPrice']
            print('Updated sell stop is: ', sell_stop_price)

In [ ]:
# Run bot function
def run_bot():
    print(f"Fetching new bars for {datetime.now().isoformat()}")
    
    # Get ohlcvc data
    bars = exchange.fetch_ohlcvc('LUNA/USD:USD', timeframe='1m', limit=50) 
    
    # Create df of our candles
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'c'])
    
    # Change timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Assign indicator used for signals and stop loss
    df = df.assign(ema = ema(df))
    
    # Apply trigger function to bot
    trigger(df)

run_bot()

"""
Schedule bot to fetch new data every minute. This can be changed to fit your trading strategy
but 1min is best for testing that your code actually works properly.
"""
schedule.every(1).minutes.do(run_bot)
while True:
    schedule.run_pending()
    time.sleep(1)